## Send an Email with Together AI and Composio

In [4]:
!pip install -qU composio_core composio-togetherai

In [22]:
!composio add gmail

> Redirecting you to the login page
> https://app.composio.dev/?cliKey=421a4d27-77cd-46b5-8ae8-03e8a786bf92
> Enter authentication code: ^C
Aborted!


ADD IMAGE OF THE ADDING PROCESS

In [ ]:
import os
from composio_togetherai import ComposioToolSet, Action
from together import Together


client = Together(api_key=os.getenv("TOGETHER_API_KEY"))
toolset = ComposioToolSet(api_key=os.getenv("COMPOSIO_API_KEY"))

send_email_tool = toolset.get_tools(
    [Action.GMAIL_SEND_EMAIL], check_connected_accounts=False
)
response = client.chat.completions.create(
    model="Qwen/Qwen2.5-72B-Instruct-Turbo",
    messages=[
        {"role": "system", "content": "You are a head of AGI at OpenAI."},
        {
            "role": "user",
            "content": "Send an email to Sam Altman in one sentence that AGI is coming 5 years earlier.",
        },
    ],
    tools=send_email_tool,
)

response.model_dump()


/opt/anaconda3/envs/cookbook/lib/python3.12/site-packages/composio/tools/toolset.py:543: UserWarning: Not verifying connected accounts for apps. Actions may fail when the Agent tries to use them.
  warnings.warn(


{'id': 'nnSDCvx-4yUbBN-925f0ea4d8beabe2',
 'object': <ObjectType.ChatCompletion: 'chat.completion'>,
 'created': 1742912512,
 'model': 'Qwen/Qwen2.5-72B-Instruct-Turbo',
 'choices': [{'index': 0,
   'logprobs': None,
   'seed': 2462741448448379000,
   'finish_reason': <FinishReason.ToolCalls: 'tool_calls'>,
   'message': {'role': <MessageRole.ASSISTANT: 'assistant'>,
    'content': None,
    'tool_calls': [{'id': 'call_to4avrovseofen8w5tqe4stq',
      'type': 'function',
      'function': {'name': 'GMAIL_SEND_EMAIL',
       'arguments': '{"recipient_email":"sam@openai.com","subject":"AGI Timeline Update","body":"AGI is coming 5 years earlier.","is_html":false}'},
      'index': 0}]}}],
 'prompt': [],
 'usage': {'prompt_tokens': 523, 'completion_tokens': 55, 'total_tokens': 578}}

In [ ]:
#modify to send email to my_email.dev
def gmail_preprocessor(inputs: dict) -> dict:
    inputs["recipient_email"] = "my_email.dev"  # Change to an email you can access to test!
    return inputs

#remove recipient_email from schema
def gmail_schema_processor(schema: dict) -> dict:
    del schema["recipient_email"]
    return schema


processed_send_email_tool = toolset.get_tools(
    actions=[Action.GMAIL_SEND_EMAIL],
    processors={
        "schema": {Action.GMAIL_SEND_EMAIL: gmail_schema_processor},
        "pre": {Action.GMAIL_SEND_EMAIL: gmail_preprocessor},
    },
    check_connected_accounts=True,
)

In [3]:
response = client.chat.completions.create(
    model="Qwen/Qwen2.5-72B-Instruct-Turbo",
    messages=[
        {"role": "system", "content": "You are a head of AGI at OpenAI."},
        {
            "role": "user",
            "content": "Send an email to Sam Altman in one sentence that AGI is coming 5 years earlier.",
        },
    ],
    tools=processed_send_email_tool,
)

exec_response = toolset.handle_tool_calls(response)


In [4]:
exec_response

[{'data': {'response_data': {'id': '195cdaf5cdfedf06',
    'threadId': '195cdaf5cdfedf06',
    'labelIds': ['UNREAD', 'SENT', 'INBOX']}},
  'error': None,
  'successfull': True,
  'successful': True}]